# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [22]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [23]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [24]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        if hyperparams.get('pca_standardize', False):
            pca_scaler = StandardScaler()
            X_train = pca_scaler.fit_transform(X_train)
            X_val = pca_scaler.transform(X_val)
            
        n_componenti = hyperparams['pca_components']
        pca = PCA(n_componenti)
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)


    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati
    vet_uno_train = np.ones((X_train.shape[0], 1))
    X_train = np.c_[vet_uno_train,X_train]

    vet_uno_validation = np.ones((X_val.shape[0], 1))
    X_val= np.c_[vet_uno_validation,X_val]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']: #regressione con regolarizzazione
        lambda_r = hyperparams['reg_lambda']
        matrice_identica = np.eye(X_train.shape[1])
        matrice_identica[0,0] = 0
        parametri = np.linalg.inv(X_train.T @ X_train + lambda_r * matrice_identica) @ X_train.T @ y_train
    else: #regressione senza regolarizzazione
        parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

    # Calcolo predizioni
    y_pred_training = X_train @ parametri
    y_pred_validation = X_val @ parametri

    # Calcola il MAE
    MAE_train = np.mean(np.abs(y_train - y_pred_training))
    MAE_val = np.mean(np.abs(y_val - y_pred_validation))
    
    return MAE_train, MAE_val

# Suddividi i dati in training, validation e test set
percent_train = 0.7
percent_val = 0.15
percent_test = 0.15

n_tot, colonne = X.shape
fine_train = int(percent_train * n_tot)
fine_val = fine_train + int(percent_val * n_tot) 

# print("Fine train:", fine_train)
# print("Fine val:", fine_val)

X_train = X[:fine_train]
y_train = y[:fine_train]

X_val = X[fine_train:fine_val]
y_val = y[fine_train:fine_val]

X_train_e_val = X[:fine_val]
y_train_e_val = y[:fine_val]

X_test = X[fine_val:]
y_test = y[fine_val:]

# print("Shape X_train:", X_train.shape)
# print("Shape X_val:", X_val.shape)
# print("Shape X_test:", X_test.shape)

# Trova la configurazione di iperparametri migliore
min_mae_val = float('inf')
min_mae_train = None
config_trovata = None

for config in configs:
    mae_train, mae_val = pipeline(X_train.copy(), y_train.copy(), X_val.copy(), y_val.copy(), config)
    if mae_val < min_mae_val:
        config_trovata = config
        min_mae_val = mae_val
        min_mae_train = mae_train

# Riallena il modello sul training set completo
def test(X_train_e_val, y_train_e_val, X_test, y_test, config):
    X_train = X_train_e_val.copy()
    X_test = X_test.copy();

    if config['use_pca']:
        if config.get('pca_standardize', False):
            pca_scaler = StandardScaler()
            X_train = pca_scaler.fit_transform(X_train)
            X_test = pca_scaler.transform(X_test)
        n_componenti = config['pca_components']
        pca = PCA(n_componenti)
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

    if config['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    vet_uno_train = np.ones((X_train.shape[0], 1))
    X_train = np.c_[vet_uno_train,X_train]
    
    vet_uno_test = np.ones((X_test.shape[0], 1))
    X_test = np.c_[vet_uno_test,X_test]

    if config['use_regularization']: #regressione con regolarizzazione
        lambda_r = config['reg_lambda']
        matrice_identica = np.eye(X_train.shape[1])
        matrice_identica[0,0] = 0
        parametri = np.linalg.inv(X_train.T @ X_train + lambda_r * matrice_identica) @ X_train.T @ y_train_e_val
    else: #regressione senza regolarizzazione
        parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train_e_val

    y_pred_test = X_test @ parametri
    MAE_test = np.mean(np.abs(y_test - y_pred_test))
    return MAE_test
        
# Calcola il MAE sul test set
MAE_test = test(X_train_e_val, y_train_e_val, X_test, y_test, config_trovata)

# Stampa  risultati
print("Configurazione migiore: ")
print(config_trovata)
print("MAE train: ")
print(min_mae_train)
print("MAE validation: ")
print(min_mae_val)
print("MAE test: ")
print(MAE_test)

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Configurazione migiore: 
{'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}


AttributeError: 'numpy.float64' object has no attribute 'toString'